In [ ]:
# Unconmenti this line and replace 'key.txt' with your key
# import os
# os.environ["OPENAI_API_KEY"] = open("key.txt").read().strip() # run this code before engaging LLM model
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent, AgentExecutor # the tools needed to create a sort of chain with directives
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool
from typing import List, Dict, Any, Optional

# === TOOL DEFINITIONS ===
def search_calendar_events(title: Optional[str]=None, datetime_range_lowerbound: Optional[str]=None) -> List[str]:
    ""
    return [f"Meeting: {title or 'Team Sync'} scheduled at {datetime_range_lowerbound or 'Tomorrow 10 AM'}"]

def search_spotify_podcasts(topic: str) -> List[str]:
    return [f"Podcast on {topic}: 'Mindful Moments'", f"Podcast on {topic}: 'Daily Focus'"]

def get_stock_history(ticker: str, range: Optional[str] = "1d") -> Dict[str, Any]:
    return {"ticker": ticker, "range": range, "price_change": "+2.3%", "trend": "upward"}

calendar_tool = StructuredTool.from_function( # from_function allow the model understand the parameters of the tool, without writing them by hand
    name='search_calendar_events',
    func=search_calendar_events,
    description='Search calendar events with optional filters'
)

spotify_tool = StructuredTool.from_function(
    name='search_spotify_podcasts',
    func=search_spotify_podcasts,
    description='Search Podcasts'
)

stock_history_tool = StructuredTool.from_function(
    name='get_stock_history',
    func=get_stock_history,
    description='Search Stock data'
)

TOOLS = [calendar_tool,spotify_tool,stock_history_tool]


System_message = f"""
You are an intelligent multi-domain assistant capable of understanding natural language and switching seamlessly between tasks.
Your primary goal is to help the user by reasoning through the conversation, deciding when to use available tools, and responding in a natural and structured way.

---

### 🔧 TOOL USAGE GUIDELINES

* **Always prefer using a tool when** the user’s request clearly maps to one of the available tools.
* **Never fabricate tool arguments** — use only information explicitly provided by the user or logically inferred from prior context.
* **Confirm intent** before performing high-impact or irreversible actions (e.g., scheduling, deleting, sending).
* **If multiple tools could apply**, select the most relevant one and explain your reasoning briefly in natural language before using it.
* If the user’s request cannot be handled by any tool, provide a helpful natural-language response instead of forcing tool use.

---

### 💬 CONVERSATION STRUCTURE

For every user input:

1. **Interpret** the user’s intent and reasoning clearly.
2. **Confirm or clarify** if the request might have multiple meanings.
3. **Use the appropriate tool** with accurate arguments.
4. **Summarize results naturally**, integrating the tool’s output into a conversational, user-friendly message.
5. **Offer a helpful follow-up** (next possible action, suggestion, or confirmation).
6. **Show the assistent plan** -> BEFORE calling any tool: produce a short HUMAN-FACING plan sentence prefixed with "assistant_plan:" describing what you will do next (example: "assistant_plan: lets add this Event in the calender").
7. **Show summary** -> AFTER tool outputs, produce a clear HUMAN-FACING summary prefixed with "assistant_summary:" describing the result and next step (example: "assistant_summary: Event successfully added!").

---

### 🧩 TASK SWITCHING

* Detect when the user **changes topic or intent**. Treat each new topic as an independent sub-task.
* Maintain context from prior tasks only if it is explicitly relevant.
* Avoid confusion between separate tasks (e.g., don’t mix calendar and finance intents).
* Keep tone and style consistent across task changes.

---

### 🤖 RESPONSE STYLE

* Be concise, polite, and natural — like a capable human assistant.
* Use plain language, even when summarizing tool outputs.
* Always ground your responses in verified data or tool results.
* Use bullet points or short paragraphs for clarity when listing results.
* Never expose internal reasoning or JSON tool calls to the user. The only 'Thinking' you will output are 'assistant_plan' and 'assistant_summary'.

---

### ⚠️ ERROR HANDLING

* If a tool call fails or lacks required parameters, ask the user for clarification rather than guessing.
* If the user’s input is ambiguous, ask a confirming question.
* If the same user message includes multiple intents, handle them one by one — acknowledge completion before switching.

---

### 🧭 OVERALL OBJECTIVE

Act as a **cohesive, reasoning-driven agent** capable of:

* identifying intent,
* executing the right tool safely,
* managing task switches gracefully,
* and keeping the conversation clear, consistent, and helpful.

"""


# create prompt

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(System_message), # The systeMessage
    MessagesPlaceholder(variable_name="chat_history"), # chat_history (niettare contenuti dinamici nel prompt.)
    # MessagesPlaceholder(variable_name="system_context"), # system_context (niettare contenuti dinamici nel prompt.)
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad") # 'agent_scratchpad' È il meccanismo chiave per il reasoning step-by-step del modello.
    # Contiene i risultati intermedi (tool call + return) della sessione corrente, cioè:
    # Grazie ad esso il modello apprende anche dal 'return' delle tool invocate, e non solo dalle systemMessage.
    # Per esempio, in questo caso quando invochiamo la search tool, il return informa il modello che "wifi deve essere on e lower_setting_state deve essere off"
    # e grazie a questo return informativo(che ricordiamo non deve essere troppo lungo), l'agent_scratchpad aiuta il modello a capire di agiornare i settings prima di invocare 
    # nuovamente il 'search tool'. 
    # In parole semplici: 
    # "il modello “vede” che la causa dell’errore è il Wi-Fi spento e può decidere da solo di chiamare set_system_settings() per sistemarlo.""
])


# Crea la classe SafeConversationMemory
from langchain.schema import BaseMessage, HumanMessage

class SafeConversationMemory(ConversationBufferMemory): 
    """
    questa classe estende ConversationBufferMemory, con lo scopo di renderla piu' “intelligente” sicura.
    Enhanced memory that:
    - Cleans malformed message content (list/dict -> string)
    - Keeps track of system_settings status
    - Add the actual status into the model through context by adding it in memory, so in the systemMessage you tell the model to verify in memory to make clever decisions 
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        print('Hit self, **kwargs')
       
        # self._state_actions = {'Key_Status':{ # default status. Per far capire al modello che deve updatare questi date per accedere a serch tool
        #     'wifi':False,
        #     'low_battery_mode':True
        # }} 


    # ============================================================
    # 🧠 1️⃣ Methods to manage pending actions (dont need those for this task)
    # ============================================================

    def set_state_action(self, name: str, key:str, value:str|bool):# ( data: self, name: str, data:dict) # we call this def from the tool def create_reservation(name,data)
        """Store a pending action waiting for user confirmation."""
        # self._state_actions[name] = data
        self._state_actions[name][key] = value # queste value e key saranno sempre le stesse 2('wifi' e 'low_batteri_mode') e per questo non si sommeranno, ma cambieranno solo le value (true or false) 
        # Vengono aggiornate uno alla volta, in ordine del modello quando invoka l'update delle key value nella tool set_system_settings

    def get_state_action(self, name: str): # controls if there are any pending actions when the model calls add_to_calender(name) Tool
        """Retrieve a specific pending action."""
        return self._state_actions.get(name) # invocando questa funzione avremo l attuale status delle key 'wifi' e 'low_batteri_mode'

    # def clear_pending_action(self, name: str = None): # no need to use this function for now
    #     """Clear one or all pending actions."""
    #     if name:
    #         self._state_actions.pop(name, None)
    #     else:
    #         self._state_actions.clear()

    # ============================================================
    # 🧹 2️⃣ Override: cleaning + injecting context for the model
    # ============================================================

    def load_memory_variables(self, inputs):
        """Return cleaned chat history + pending actions as context."""
        raw = super().load_memory_variables(inputs) # il nome del method della classe ConversationBufferMemory, che qui estendiamo e modifichiamo per fare 
        # pulizie e aggiungere system_context in formato BaseMessage. (deve avere questo nome load_memory_variables)

        print('hit load_memory_variables and inputs :',inputs) # {'input': 'where is googleplex?'}
        # ---- Clean malformed messages (lists/dicts converted to strings)
        if "chat_history" in raw:  # chat_history e' la key che abbiamo dato al nostro memoryKey quando abbiamo definito 'memory' e nel prompt
            cleaned = []
            print('hit chat_history')
            for msg in raw["chat_history"]:
                if isinstance(msg, BaseMessage): # BaseMessage(HumanMessage(content:'user_input') and AIMessage(content:'ai response')) is the rigth type for our model. Its the type the model require to work without errors.
                    # It is like that:
                    # {'chat_history': [HumanMessage(content='I need a reservation', additional_kwargs={}, response_metadata={}), AIMessage(content='Could you please clarify what type of reservation you need? For example, is it for a restaurant, hotel, or something else? Additionally, please provide any specific details like the date, time, and number of people if applicable.', additional_kwargs={}, response_metadata={})}
                    # so basically our memory stored in ConversationBufferMemory is stored in this way showed in HumanMessage and AIMessage
                    msg.content = str(msg.content) # turn (list, dict) content in str type to not throw errors
                    cleaned.append(msg)
                elif isinstance(msg, str): # if there are some 'fully string type' value that can cause errors(like not structured baseMessage data), instead of BaseMessage structure in our
                      # ConversationBufferMemory chat_history , then turn this string  to HumanMessage(). This 'string' bug can hapend when the model calls a tool, creating a 
                      # 'string with the name of the tool invoked', that the Model doesnt support and cannot parse.
                      #(HumanMessage is part of baseMessage for related to the user_input. there is the exemple right up in 'if isinstance(msg, BaseMessage)'). 
                     cleaned.append(HumanMessage(content=msg)) 
            raw["chat_history"] = cleaned # puliamo l object chat_history da eventuali errory
            print('check raw ->', raw)

        # ---- Add pending actions as extra conversational context. Questo previene chiamate al tool "reservation" quando invokiamo il tool add_to_calender
        # inseriamo in memoria che c'e un pending tool ogni volta che il tool "reservation" viene invocato, e nel systemMessage diciamo specificamente
        # al modello di non invocare nuovamente il tool "reservation" quando decidiamo di mettere questo "reservatio event" come evento nel tool "calender"
        # infine ogni volta che il modello inserisce l'evento sul tool calender, il pendig state ritorna vuoto, cosicche' il processo si ripete e 
        # non ci ripeta lo stesso problema
        # if self._state_actions: # if _state_actions not empty, cioe' se ci sono dei pending_state 
        #     context_str = (
        #         f"(State actions currently stored: {self._state_actions})" # questo va a' finire nella memory nel formato humaMessage come bediamo sotto
        #     )
        #     print('check state_actions ->', context_str)
        # else:
        #     context_str = "(No state actions stored.)"
        
        # # aggiungiamo pending_actions in formato HumanMessage
        # raw["system_context"] = [HumanMessage(content=context_str)] # aggiungiamo un messaggio extra che contiene i dati di ogni turn delle values dei settngs
        # sotto il prefisso 'system_context'(inventato da me) che poi inseriamo nel prompt/AIMessagePlaceHolder, cosi ad ogni turno il modello tiene 
        # conto dei dati ogiornati o meno di settings del turno precedente
        # Lo stesso vale per chat_history
        # trasformiamo il pending action in uno HumanMessage in modo che il formato sia
        # corretto da inserire nella memory e da essere compatimile con il formato del modello.
        # Ora  Il modello “vede” il pending_action nel contesto e può comportarsi di conseguenza (nei punti 5 e 6 del systemMessage diciamo al
        # di guardare i pending_state della memory e cosa fare se ce ne sono di attivi. 
        
        return raw  # ritorniamo la struttura di ConversationBufferMemory() piu' pulita e' con l aggiunt di pending_actions


# Create llm
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)


# === MEMORY & AGENT SETUP ===
memory = SafeConversationMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input"
)

agent = create_openai_functions_agent(
    llm=llm,
    tools=TOOLS,
    prompt=prompt  # il prompt include le regole di "Assistant Responses" e "Conversation Structure"
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=TOOLS,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
)

# 3) - BEFORE calling any tool: produce a short HUMAN-FACING plan sentence prefixed with "assistant_plan:" describing what you will do next (example: "assistant_plan: i'll check the device settings and enable Wi-Fi if needed, then search for a location").

# 4) - AFTER tool outputs or after state changes, produce a clear HUMAN-FACING summary prefixed with "assistant_summary:" describing the result and next step (example: "assistant_summary: Wi-Fi was off → I turned it on. Now I'll retry the search.").

# 6) - NEVER autput chain-of-thought. The plan and summary are the only "thinking" visible to the user.




In [ ]:
# user_inputs = [
#     "Aggiungi una riunione per domani alle 10",
#     "Cerca un podcast rilassante da ascoltare",
#     "Come sta andando TSLA oggi?"
# ]

# def search_calendar_events(title: Optional[str]=None, datetime_range_lowerbound: Optional[str]=None) -> List[str]:

user_input = "Add a meeting for tomorrow at 10" 
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"]) # output where you get the answers

In [ ]:

user_input = "Is the weather good tomorrow in italy?" 
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"]) 

In [ ]:

user_input = "ok, the purpose is to talk with Erik" 
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"]) 

In [ ]:

user_input = "yes, i ll meet him at restaurant" 
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"])

In [ ]:

user_input = "Please find me a relaxing podcast to listen" 
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"])

In [ ]:
user_input = "tnx alot" 
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"])

In [ ]:
user_input = "by the way, do not modify the existing meeting" 
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"])

In [ ]:
user_input = "Hoe are you by the way?" 
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"])